In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, callbacks

# Reproductibilité
tf.keras.utils.set_random_seed(42)

In [ ]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_PATH = '/content/drive/MyDrive/ModelCNN_WEBIA.keras'
except ImportError:
    SAVE_PATH = 'ModelCNN_WEBIA.keras'

print("Model will be saved to:", SAVE_PATH)


In [ ]:
builder = tfds.builder("eurosat")

In [ ]:
# Constants configuration
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 64

print("Loading EuroSAT dataset...")

# 1. Dataset Loading (RGB version)
ds, info = tfds.load("eurosat/rgb", split="train", with_info=True, as_supervised=True)
NUM_CLASSES = info.features["label"].num_classes

# 2. Global Shuffle (Crucial before split)
ds = ds.shuffle(10000, seed=42, reshuffle_each_iteration=False)

# 3. Calculate Split Sizes (70/15/15)
total_examples = info.splits["train"].num_examples
n_train = int(0.7 * total_examples)
n_val = int(0.15 * total_examples)
n_test = total_examples - n_train - n_val

print("Total:", total_examples, "| Train:", n_train, "| Val:", n_val, "| Test:", n_test)

# 4. Split datasets
train_raw = ds.take(n_train)
rest = ds.skip(n_train)
val_raw = rest.take(n_val)
test_raw = rest.skip(n_val)

# 5. Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(label, tf.int32)
    return image, label

# 6. Build data pipelines
train_ds = (train_raw
            .map(preprocess, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTOTUNE))

val_ds = (val_raw
          .map(preprocess, num_parallel_calls=AUTOTUNE)
          .batch(BATCH_SIZE)
          .cache()
          .prefetch(AUTOTUNE))

test_ds = (test_raw
           .map(preprocess, num_parallel_calls=AUTOTUNE)
           .batch(BATCH_SIZE)
           .cache()
           .prefetch(AUTOTUNE))

# 7. Data Augmentation Layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="data_augmentation")

print("Data pipeline ready.")

In [ ]:
def residual_block(x, filters, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, (3, 3), strides=stride, padding="same",
                      use_bias=False, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('gelu')(x)

    x = layers.Conv2D(filters, (3, 3), padding="same",
                      use_bias=False, kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    if stride > 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding="same",
                                 use_bias=False, kernel_initializer='he_normal')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('gelu')(x)
    return x


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")

x = data_augmentation(inputs)
x = layers.Conv2D(64, (3, 3), padding="same", use_bias=False, kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('gelu')(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Small ResNet-like stack
x = residual_block(x, 64, stride=1)
x = residual_block(x, 64, stride=1)

x = residual_block(x, 128, stride=2)
x = residual_block(x, 128, stride=1)

x = residual_block(x, 256, stride=2)
x = residual_block(x, 256, stride=1)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="predictions")(x)

model = models.Model(inputs, outputs, name="ModelCNN_WEBIA")

my_callbacks = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=150,
    callbacks=my_callbacks
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Créer une figure avec deux sous-graphiques côte à côte
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graphe de la perte
axes[0].plot(history.history['loss'], label='Train Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Graphe de l'accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_title('Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

model.save(SAVE_PATH)
print("Saved model to:", SAVE_PATH)